In [ ]:
######Brian Snyder 10/8/19  gFOD 15-Minute Sustained Wind Speed Forecast
# Gets the next 2 hours forecast15 minutes apart.  There's up to 7 hours available, but that's too much for the entire globe.  Need to add boundry boxes if all 7 days are needed.
# Total size is around 1G for all 7 days for the globe.

import requests,json,datetime # need to install these libraries to run
from arcgis.gis import GIS
from arcgis import features
from arcgis.features import FeatureLayerCollection

##All TWC visual products have a product name and number
prod_num=str(59) # gFOD 15-Minute Sustained Wind Speed Forecast
prod_name='Windspeedheightaboveground' # Per TWC naming conventions
APIKEY='YOUR_WEATHER_COMPANY_WEATHER_KEY_HERE'  #Enter your API key 


########################PRODUCT INVENTORY CALL################################
#We get the most recent runtime.
def InvCall(timeSlice):
    base1 = 'https://api.weather.com/v2/tiler/info?products='+prod_num+':'+prod_name
    base2 = '&apiKey='+APIKEY
    url = base1 + base2
    print("Call #1: " + url)
    APIcall = requests.get(url) #Our API Call     to grab the latest forecast
    if APIcall.status_code == 200:  #if(APIcall.ok):
        data = APIcall.json() 
        runtime = data["layers"][prod_num][prod_name]["dimensions"][0]["rt"][0]
        timeSliceRemaining = 27 - timeSlice
        timestamp = data["layers"][prod_num][prod_name]["dimensions"][0]["t"][timeSliceRemaining]            
        return [runtime, timestamp]

##########################   API CALL   ###################################
def APIcall(runtime, timestamp,dateTime, timeslice, esriId): # function for pulling data 
    count=0
    num_layers = 6     # 6 layers > 10, 20, 30, 40 ,50 MPH 
    inc=2.235           # Each increment is 2.235 m/s = 5 MPH
    low_limit = 4.47   # Thresholds are in m/s 2.235 m/s = 5 MPH.  Starting with 10 MPH.
    low_limitS = str(low_limit)
    high_limit=low_limit+5
    high_limitS = str(high_limit)      
                 
    base1 = 'https://api.weather.com/v2/featurizer/feature?product='+prod_num+':'+prod_name
    base2 = '&rt=' + runtime + '&t='+timestamp + '&apiKey='+APIKEY+'&geometryType=polygon'       
    fc = {
            "type": "FeatureCollection",
            "features": []
        }
    for x in range(0,num_layers):        
        count=count+1
        base3 = '&threshold='+low_limitS

        url = base1 + base2 + base3  # make API URL   
        print ("Call #2: " + url) 
        data = requests.get(url).json() #Our API Call
        
        for y in data['features']:           
            f = {"type": "Feature", "properties": {}, "geometry": None}            
            f['geometry'] = y['geometry']            
            fifteenMinuteSustainedWindSpeedForecast = float(y['properties']['threshold'])
            fifteenMinuteSustainedWindSpeedForecastMPH = round(fifteenMinuteSustainedWindSpeedForecast/0.44704, 0) #convert m/s to MPH by dividing m/s by 0.44704
            f['properties']['Sustained Wind Speed Forecast'] = str(fifteenMinuteSustainedWindSpeedForecastMPH)
            f['properties']['Date'] = dateTime            
            fc['features'].append(f)

        #reset limits
        low_limit=low_limit+inc
        low_limitS=str(low_limit)                  
        high_limit=high_limit+inc
        high_limitS=str(high_limit)

    #Update the layer now that multiple layers have been combined.
    with open("gFOD_15_Minute_Sustained_Wind_Speed_Forecast_MPH_" + str(timeslice) + ".geojson", "w") as outfile:
        json.dump(fc, outfile)
    if(len(fc['features']) != 0):
        portalURL = "http://ibmmain.maps.arcgis.com"
        userID = "ESRI_USERNAME_HERE"
        password = "ESRI_PASSWORD_HERE"
        gis = GIS(portalURL, userID, password)
        existing_ID = esriId  # Map ID
        wx_item = gis.content.get(existing_ID)
        collection = features.FeatureLayerCollection.fromitem(wx_item)
        collection.manager.overwrite('gFOD_15_Minute_Sustained_Wind_Speed_Forecast_MPH_' + str(timeslice) + '.geojson')  
        print("Updating Layer: gFOD_15_Minute_Sustained_Wind_Speed_Forecast_MPH_" + str(timeslice) + ".geojson")
    else:
        print("NOT Updating Layer: gFOD_15_Minute_Sustained_Wind_Speed_Forecast_MPH_" + str(timeslice) + ".geojson since there is nothing to update")

################################################################################
esriIds = ["3e6dfde8ee0e46dd88f5abac18975c7c", "5469801ecce44e8c9e46e0d8ab88f418", "c057f211826f4ca8b3761b100bcd8a2c", "591aa078fe854a768ece527aa3ec1fac","7cc090f4b3a245bf8dcdc25827f2ed98", "533fac44375d4954b76d029acaf8ce6c", "1b582e8015f344b1aa8587a49b5e1cfa", "77b42e4c6d124ff1bd00c07876269746"]
for x in range(0,8):
    timeInfo = InvCall(x)  ## Grab time stamp    
    print(timeInfo)
    fmt = "%Y-%m-%d %H:%M:%S"
    # local time
    #t = datetime.datetime.fromtimestamp(int(timeInfo[1])/1000)
    #print (t.strftime(fmt)) # prints 2012-08-28 02:45:17
    # utc time
    t_utc = datetime.datetime.utcfromtimestamp(int(timeInfo[1])/1000)
    t_utc_formatted = t_utc.strftime(fmt)
    print (t_utc_formatted)    
    APIcall(timeInfo[0],timeInfo[1], t_utc_formatted, x, esriIds[x])